# Import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
import sys
import os
import pickle

# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Load environment variables from .env file
load_dotenv()
from src.config_loader import config
from src.generate import ( 
    add_practice_phrases,
    create_story_plan_and_dialogue,
    add_translations,
    add_audio,
    create_album_files)
from src.utils import save_defaultdict, generate_story_image, create_html_story
from src.anki_tools import export_to_anki

## Setup Google Cloud credentials and prerequisites
You will need a Google Project with the following APIs enabled:
* Text to Speech
* Translate
* Vertex AI with the following Anthropic models enabled (from the model garden)
    * Sonnet 3.5
    * Imagen 3
* Add your GOOGLE_PROJECT_ID to the .env file

You should alter src/config.json which contains your target language.


In [ ]:
from google.auth import default
credentials, project = default()

### Setup story parameters

In [ ]:
STORY_NAME = "lost in stockholm" # this phrase will be used to prompt the story creation and the cover image creation

STORY_NAME_CLEAN = STORY_NAME.replace(" ", "_") #just to make directory names easier
OUTPUT_DIR = f"../outputs/{STORY_NAME_CLEAN}"
STORY_DATA_PATH = f"{OUTPUT_DIR}/story_data_{STORY_NAME_CLEAN}.json"
# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
config._load_config()
print(f"Your target language is {config.TARGET_LANGUAGE_NAME}")

### Generate story content and outputs

In [ ]:

story_data_dict = create_story_plan_and_dialogue(STORY_NAME, output_dir=OUTPUT_DIR)
save_defaultdict(story_data_dict, STORY_DATA_PATH)


In [ ]:

story_data_dict = add_practice_phrases(story_data_dict)
save_defaultdict(story_data_dict, STORY_DATA_PATH)


story_data_dict = add_translations(story_data_dict)
save_defaultdict(story_data_dict, STORY_DATA_PATH)
story_data_dict = add_audio(story_data_dict)


#Saving as pickle file to retain audio data
with open(F'{STORY_DATA_PATH}.pkl', 'wb') as file:
    pickle.dump(dict(story_data_dict), file)

IMAGE_DATA = generate_story_image(STORY_NAME_CLEAN)

with open(f"{OUTPUT_DIR}/cover_artwork.jpg", "wb") as f:
    f.write(IMAGE_DATA)


create_album_files(story_data_dict, IMAGE_DATA, OUTPUT_DIR, STORY_NAME_CLEAN)


create_html_story(
        story_data_dict,
        f"{OUTPUT_DIR}/{STORY_NAME_CLEAN}.html",
        component_path="../src/StoryViewer.js",
        title=STORY_NAME,
    )

#export_to_anki(story_data_dict, OUTPUT_DIR, STORY_NAME_CLEAN)


### If picking up from a saved file:

In [ ]:
with open(F'{STORY_DATA_PATH}.pkl', 'rb') as file:
    story_data_dict = pickle.load(file)

In [ ]:
from src.utils import load_json


story_data_dict = load_json(STORY_DATA_PATH)

In [ ]:
story_data_dict